In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Extract the image tar files
!tar -xf "/content/drive/MyDrive/Graduation Project/AffectNet/train_images.tar" -C "/"
!tar -xf "/content/drive/MyDrive/Graduation Project/AffectNet/val_images.tar" -C "/"

In [ ]:
# Define train data path
train_annotations = "/content/drive/MyDrive/Graduation Project/AffectNet/train_annotations.csv"
train_images = "/content/train_set/images"

# Define val data path
val_annotations = "/content/drive/MyDrive/Graduation Project/AffectNet/val_annotations.csv"
val_images = "/content/val_set/images"

In [ ]:
# Import dependencies
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision.transforms import Lambda
from torchvision.transforms import v2

In [ ]:
# Custom Dataset Class
class AffectNet(Dataset):
    def __init__(self, annotations_file, img_root_dir, transform=None):
        self.annotations = pd.read_csv(annotations_file)
        self.root_dir = img_root_dir
        self.transform = transform

        # Check if number of images and annotations match
        if len(self.annotations) != len(os.listdir(self.root_dir)):
            raise ValueError(f"Number of images and annotations do not match:\
            {len(self.annotations)} != {len(os.listdir(self.root_dir))}"
                             )

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        # Get image name and create path
        img_name = f"{self.annotations.iloc[idx, 0]}.jpg"
        img_path = os.path.join(self.root_dir, img_name)

        # Read image
        image = read_image(img_path)

        # Get labels and convert to tensor
        labels = self.annotations.iloc[idx, 1:]
        labels = labels.to_numpy()
        labels = torch.from_numpy(labels.astype('float'))

        # One-Hot Encode the categorical expressions
        ohe = Lambda(lambda y: torch.zeros(8, dtype=torch.float).scatter_(0, torch.tensor(int(y)), value=1))
        exp = ohe(labels[2])
        labels = torch.cat([labels[:2], exp], dim=0)

        # Apply input transforms
        if self.transform:
            image = self.transform(image)

        # Return image and labels
        return image, labels

In [ ]:
torch.manual_seed(42)

val_data = AffectNet(val_annotations, val_images)
val_dataloader = DataLoader(val_data, batch_size=16, shuffle=True)